In [ ]:
import neologdn
import os
import urllib.request
from matplotlib.font_manager import FontProperties
import MeCab
from __future__ import unicode_literals
import re
import unicodedata
import sagemaker
from sagemaker import get_execution_role
import boto3
import json
import pandas as pd
import warnings
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import codecs
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#chaneg dummies to matrix
from scipy.sparse import csr_matrix,coo_matrix

import multiprocessing

from multiprocessing import Pool

import pickle

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout, Input, concatenate
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Model
import os
import tarfile
import string
import numpy as np



In [ ]:
#tokenize japanase words
def wakati_by_mecab(text):
    tagger = MeCab.Tagger('-Ochasen -d /usr/lib64/mecab/dic/mecab-ipadic-neologd')
    tagger.parse('') 
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        pos = node.feature.split(",")[0]
        #if pos in ["名詞","動詞","形容詞","記号"]:   # 対象とする品詞
        if pos in ["名詞","動詞","形容詞"]:   # 対象とする品詞
            word = node.surface
            word_list.append(word)
        node = node.next
    return " ".join(word_list)

In [ ]:
def remove_chars(row):
    cust_list=['\d+','■','\n','・','◇','①','②','③','④','【】','】','【','☆']
    del_list = string.ascii_letters + string.punctuation
    for i in del_list :
        row = row.str.replace(i,'')
    for i in cust_list :
        row = row.str.replace(i,'')
    return row

In [ ]:
file_path="/home/ec2-user/SageMaker/Notebooks_For_CX_Usecases/Sentiment_Analyzer/raw_data/tokenized_amazon_comment.pkl"
with open(file_path, 'rb') as pickle_file:
    df_amazon = pickle.load(pickle_file)

In [ ]:
file_path="/home/ec2-user/SageMaker/Notebooks_For_CX_Usecases/Sentiment_Analyzer/raw_data/tokenized_mixed_comment.pkl"
with open(file_path, 'rb') as pickle_file:
    df_mixed = pickle.load(pickle_file)

In [ ]:
df_mutilabel=df_mixed.append(df_amazon)
df_mutilabel = df_mutilabel.sample(frac=1).reset_index(drop=True)

In [ ]:
train_texts = df_mutilabel["comment"].values
train_labels = df_mutilabel["label"].values


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

text=list(df_mutilabel['comment'].values)


top_vocab=25000

tokenizer = Tokenizer(top_vocab)
#list xは上に書いたようにテキストデータが格納されている
tokenizer.fit_on_texts(text)

maxlen_value=100

# 对每一句影评文字转换为数字列表，使用每个词的编号进行编号
x_train_seq = tokenizer.texts_to_sequences(train_texts)
#x_test_seq = tokenizer.texts_to_sequences(test_texts)
x_train = sequence.pad_sequences(x_train_seq, maxlen=maxlen_value)
#x_test = sequence.pad_sequences(x_test_seq, maxlen=maxlen_value)
y_train = np.array(train_labels)
#y_test = np.array(test_labels)
with open('tokenizer/SA_CX_LSTM-GRU-Stack_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
import copy
x = list(tokenizer.word_counts.items())
s = sorted(x,key=lambda p:p[1],reverse=True)
word_index = tokenizer.word_index # 不受vocab_size的影响
small_word_index = copy.deepcopy(word_index) # 防止原来的字典也被改变了
print("Removing less freq words from word-index dict...")
for item in s[top_vocab:]:
    small_word_index.pop(item[0])
print("Finished!")
print(len(small_word_index))
print(len(word_index))

In [ ]:
import gensim
#配布されていたfastTextの次元数は300だったので300
EMBEDDING_DIM = 200
#embedding_metrix = np.zeros(top_vocab+1,EMBEDDING_DIM)
embedding_matrix = np.random.uniform(size=(top_vocab+1,EMBEDDING_DIM))
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/home/ec2-user/SageMaker/entitiy_vector_dic/entity_vector/entity_vector.model.bin', binary=True)
for word, i in small_word_index.items():
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        #print("Word: [",word,"] not in wvmodel! Use random embedding instead.")
        pass

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# AUC for a binary classifier
import tensorflow as tf
from keras import backend as K

def auc(y_true, y_pred):   
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)

#-----------------------------------------------------------------------------------------------------------------------------------------------------
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)    
    return FP/N
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)    
    return TP/P

In [ ]:
from keras import backend as K
from keras.models import Model,load_model
from keras.layers import Dense, Embedding, Input, concatenate, Flatten, SpatialDropout1D
from keras.layers import CuDNNLSTM,LSTM, Bidirectional, GlobalMaxPool1D, Dropout, CuDNNGRU,GRU, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, LearningRateScheduler, Callback
from keras.optimizers import Adam, Adadelta, SGD, RMSprop, Nadam, Adamax

def get_model():
    inputs = Input(shape=(maxlen_value,))
    wv_dim=200
    vocab_size=top_vocab
    use_pretrained_wv = True
    if use_pretrained_wv:
        wv = Embedding(vocab_size+1,wv_dim,input_length=maxlen_value,weights=[embedding_matrix],trainable=False)(inputs)
    else:
        wv = Embedding(vocab_size+1,wv_dim,input_length=MAXLEN)(inputs)
        
    #x = SpatialDropout1D(0.2)(wv)
    x = Bidirectional(CuDNNLSTM(256, return_sequences=True))(wv)
    x = Dropout(0.5)(x)
    x = Bidirectional(CuDNNGRU(256, return_sequences=True))(x)
    x = Dropout(0.5)(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])
    x = Dense(128, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=x)
    model.summary()
    opt = Adam(lr=2e-3)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy',auc])
    return model

In [ ]:
checkpoint_path = "models/ML_TXT_CS_LSTM-GRU-Stack-weights-improvement-{val_auc:.4f}.hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)
 
cp_callback = ModelCheckpoint(checkpoint_path,
                              save_best_only=True,
                              verbose=0)
 
callbacks = [
    EarlyStopping(patience=20, monitor='val_auc',mode='max'),
    cp_callback
]

In [ ]:
batch_size = 128
epochs = 5
m = get_model()

m.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,shuffle=True,validation_split=0.3,callbacks=callbacks,verbose=1)
